<a href="https://colab.research.google.com/github/rogeriowfbarroso/ImersaoAlura/blob/main/ProjetoImersaoAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q -U google-generativeai

In [10]:
import google.generativeai as genai
from google.colab import userdata


API_KEY=userdata.get('secret_key')
genai.configure(api_key=API_KEY)


In [11]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [21]:
title = "Olá mundo"
sample_text = (" Produção de café no Brasil é responsável por cerca de um terço da produção mundial de café, o que faz o país ser de longe o maior produtor - uma posição mantida nos últimos 150 anos. Em 2012 foram produzidas 50 milhões de sacas, totalizando 3 milhões de toneladas. Em 2009 foram produzidas 2,4 milhões de toneladas")
embeddings = genai.embed_content(model ="models/embedding-001", content=sample_text, title = title, task_type="retrieval_document")
print(embeddings)

{'embedding': [0.012199068, -0.035856213, -0.03209583, 0.022974433, 0.06105388, 0.015075699, -0.019813271, -0.06620406, 0.036346246, 0.031268187, -0.02738324, 0.0009824969, 0.013886492, -0.033424355, -0.046167776, 0.019568548, -0.010641857, 0.047299106, -0.020154394, -0.04960551, -0.03597995, -0.003933601, 0.0037744804, -0.013513043, 0.0061616288, -0.022606365, -0.02640196, -0.026865406, -0.031845037, 0.036274016, -0.061728362, 0.06806982, -0.020918746, 0.009590833, -0.018808886, -0.051616658, -0.038188282, 0.072231725, -0.022792248, 0.035103515, -0.009540817, -0.013757173, -0.050378963, 0.028288253, 0.023330195, -0.024375949, 0.0010235403, 0.015617731, 0.00015437824, -0.06327801, -0.019516218, 0.0050557596, 0.06459561, -0.0035199313, 0.018683568, -0.026529625, 0.026784867, -0.019794274, -0.040669594, 0.009025459, -0.00534748, -0.0035192529, 0.025922636, 0.02360859, -0.024765655, -0.05566596, -0.047531813, 0.019860756, 0.059658855, -0.047804352, 0.0015650669, -0.0690138, 0.020044327, -

In [19]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [22]:
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']
df

,Titulo,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [24]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [25]:
query = "How do you shift gears in the Google car?"
model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

In [26]:
def find_best_passage(query, dataframe, model):

  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [27]:
passage = find_best_passage(query, df)
passage

TypeError: find_best_passage() missing 1 required positional argument: 'model'

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
prompt = make_prompt(query, passage)
print(prompt)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)